Preprocessing ideas: 
1) Removal of not arabic 
2) Removal and saving of diacritics 
3) Segementation
4) Tokenization
5) Lemmatization

Feature Extraction ideas: 
1) POS
2) Morphological quadruples
3) Context
4) Last character


In [1]:
!pip install pyarabic

In [2]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

c:\Users\Mai\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mai\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import re
from Input_Utils import input_utils
from Preprocessing import preprocessing_utils
import pyarabic.araby as araby
from pyarabic.araby import strip_tashkeel, is_arabicrange, strip_diacritics


In [4]:
def remove_diacritics(text):
    stripped = araby.strip_diacritics(text)
    diacritics_dic = [{i: char} for i,char in enumerate(text) if char not in stripped]
    return stripped, diacritics_dic

In [5]:
def remove_non_arabic_char(text):
    regex = re.compile('[^؀-ۿ ]')
    result = regex.sub('', text)
    return result

In [6]:
train_dataset = input_utils.read_input('Dataset/train.txt')
text = train_dataset[0]
result = remove_non_arabic_char(text)
stripped, diacritics = remove_diacritics(result)

In [7]:
!pip install qalsadi
import qalsadi

In [8]:
import qalsadi.lemmatizer

In [9]:
lemmer = qalsadi.lemmatizer.Lemmatizer()
lemmas = lemmer.lemmatize_text('فعل')
lemmas

['عل']

In [10]:
!python3 -m pip install --index-url https://test.pypi.org/simple/ --no-deps FarasaPy3

Looking in indexes: https://test.pypi.org/simple/


In [15]:
import FarasaPy3
from FarasaPy3.api import FarasaPy3
farasaApi = FarasaPy3()
result = farasaApi.Segmentation(text)
print(result)

ModuleNotFoundError: No module named 'FarasaPy3'